In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with project


In [2]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds = ws.get_default_datastore()

if 'churn dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'churn-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'churn-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='telco dataset',
                                description='chrun data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Validating arguments.
Arguments validated.
Uploading file to churn-data/
Uploading an estimated of 3 files
Target already exists. Skipping upload for churn-data/.amlignore
Target already exists. Skipping upload for churn-data/.amlignore.amltmp
Target already exists. Skipping upload for churn-data/telco-csv.csv
Uploaded 0 files
Creating new dataset
Dataset registered.


In [3]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'telco_churn_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

telco_churn_pipeline


In [4]:
%%writefile $experiment_folder/prep_churn.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
churn = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(churn))
run.log('raw_rows', row_count)

# create a copy
churn2 = churn.copy()

# drop those colume have null values
churn2 = churn2.drop(['loglong','logtoll','logequi','logcard','logwire','lninc'],axis = 1)

# separating num, categorical and target
numerical = ["tenure","age","address","income","employ","reside","longmon","tollmon","equipmon","cardmon","wiremon","longten","tollten","equipten",'cardten',"wireten"]
categorical =["marital","retire","gender","voice","pager","internet","callid","callwait","forward","confer","ebill","ed","tollfree","equip","callcard","wireless","multline","region","custcat"]
target = ['churn']

# convert categorical to num
for x in churn2.columns:
    if churn2[x].dtypes == 'object':
        churn2[x] = pd.Categorical(churn[x]).codes

# Normalize the numeric columns
scaler = MinMaxScaler()
numerical = ["tenure","age","address","income","employ","reside","longmon","tollmon","equipmon","cardmon","wiremon","longten","tollten","equipten",'cardten',"wireten"]
churn2[numerical] = scaler.fit_transform(churn2[numerical])

# Log processed rows
row_count = (len(churn2))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
churn2.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

Overwriting telco_churn_pipeline/prep_churn.py


In [5]:
%%writefile $experiment_folder/train_churn.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
args = parser.parse_args()
training_data = args.training_data

# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data...")
file_path = os.path.join(training_data,'data.csv')
churn2 = pd.read_csv(file_path)

# Separate features and labels
X, y = churn2[['region', 'tenure', 'age', 'marital', 'address', 'income', 'ed',
       'employ', 'retire', 'gender', 'reside', 'tollfree', 'equip', 'callcard',
       'wireless', 'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon',
       'longten', 'tollten', 'equipten', 'cardten', 'wireten', 'multline',
       'voice', 'pager', 'internet', 'callid', 'callwait', 'forward', 'confer',
       'ebill',
       'custcat']].values, churn2['churn'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Train adecision tree model
print('Training a decision tree model...')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'churn_model.pkl')
joblib.dump(value=model, filename=model_file)

# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'churn_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})


run.complete()

Overwriting telco_churn_pipeline/train_churn.py


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-churn-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [7]:
%%writefile $experiment_folder/experiment_churn_env.yml
name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting telco_churn_pipeline/experiment_churn_env.yml


In [8]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_churn_env", experiment_folder + "/experiment_churn_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_churn_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [9]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
churn_ds = ws.datasets.get("telco dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_churn.py",
                                arguments = ['--input-data', churn_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_churn.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [10]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-churn-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Prepare Data [b9a3bbbd][34fd9dde-9082-460e-8269-91fecbdbf63b], (This step will run and generate new outputs)
Created step Train and Register Model [bc8459ae][89bd93fe-5351-4510-bbc7-55b9f11efb18], (This step will run and generate new outputs)
Submitted PipelineRun b6220201-0334-4af8-9451-0f9cf56e67cf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b6220201-0334-4af8-9451-0f9cf56e67cf?wsid=/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourcegroups/project_dp100/workspaces/project&tid=474565c1-bca4-4295-a2f5-b0c7dbf2591c
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: b6220201-0334-4af8-9451-0f9cf56e67cf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b6220201-0334-4af8-9451-0f9cf56e67cf?wsid=/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourcegroups/project_dp100/workspaces/project&tid=474565c1-bca4-4295-a2f5-b0c7dbf2591c
PipelineRun Status: Running


StepRunId: 541c59ae-3678-4792-9e1c-074311fde441
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/541c59ae-3678-4792-9e1c-074311fde441?wsid=/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourcegroups/project_dp100/workspaces/project&tid=474565c1-bca4-4295-a2f5-b0c7dbf2591c
StepRun( Prepare Data ) Status: NotStarted
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished
{'runId': '541c59ae-3678-4792-9e1c-074311fde441', 'target': 'your-churn-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2023-02-04T05:19:34.907734Z', 'endTimeUtc': '2023-02-04T05:20:02.016097Z', 'ser

'Finished'

In [11]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

Train and Register Model :
	 Accuracy : 0.6533333333333333
	 AUC : 0.5745037072470701
	 ROC : aml://artifactId/ExperimentRun/dcid.4e60bf09-1aa3-4800-b0fa-331d1e59f429/ROC_1675488023.png
Prepare Data :
	 raw_rows : 1000
	 processed_rows : 1000


In [12]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="churn-training-pipeline", description="Trains churn model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
churn-training-pipeline,32c8bc71-c1ef-4dd4-83b7-901cf328af10,Active,REST Endpoint


In [13]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://eastus.api.azureml.ms/pipelines/v1.0/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourceGroups/project_dp100/providers/Microsoft.MachineLearningServices/workspaces/project/PipelineRuns/PipelineSubmit/32c8bc71-c1ef-4dd4-83b7-901cf328af10


In [14]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Authentication header ready.


In [15]:
import requests

experiment_name = 'mslearn-churn-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

'e98f7fd7-44a7-4daf-b601-d036c4c85ecf'

In [16]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
published_pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: e98f7fd7-44a7-4daf-b601-d036c4c85ecf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e98f7fd7-44a7-4daf-b601-d036c4c85ecf?wsid=/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourcegroups/project_dp100/workspaces/project&tid=474565c1-bca4-4295-a2f5-b0c7dbf2591c
PipelineRun Status: Running


StepRunId: a71f03b8-4153-4377-a780-94b585f6177a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a71f03b8-4153-4377-a780-94b585f6177a?wsid=/subscriptions/bd5c51d3-de0b-41fc-99f7-375e6737efda/resourcegroups/project_dp100/workspaces/project&tid=474565c1-bca4-4295-a2f5-b0c7dbf2591c
StepRun( Prepare Data ) Status: NotStarted
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished
{'runId': 'a71f03b8-4153-4377-a780-94b585f6177a', 'target': 'your-churn-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2023-02-04T05:21:02.55732Z', 'endTimeUtc': '2023-02-04T05:21:25.451261Z', 'serv

'Finished'